<a href="https://colab.research.google.com/github/BecomeAllan/S2Search/blob/main/SemanticScholarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consumindo a API do SemanticScholar

A seguir, tem uma classe sem parámetros chamada `Search()`, que ao instanciar-la em uma variável é possível fazer pesquisas sobre papers utilizando a api do SemanticScholar, dentre os parâmetros temos:

- Buscar: Pesquisas sobre tópicos onde adicionar tópicos utiliza-se + (mais) e remover tópicos usamos - (menos)

- Fields: O que será retornado como dados. Para utilizar, escolha dentre as opções sem utilizar espaço e separadas de virgulas:
  - externalIds
  - url
  - title
  - abstract
  - venue 
  - year 
  - referenceCount
  - citationCount
  - influentialCitationCount
  - isOpenAccess
  - fieldsOfStudy
  - authors 

- Limite: Número de papers a ser retornados

**Obs:** A api do SemanticScholar disponibiliza 100 query's a cada 5 min.

In [ ]:
#@title Classe para pesquisa no SemanticScholar
import IPython
from google.colab import output
import pandas as pd

class Search():
  def __init__(self):
    self.data = ""

    self._start()
  
  def _start(self):

    output.register_callback('notebook.searching', self._searching)
    output.register_callback('notebook.AddListItem', self._add_list_item)
    # output.register_callback('notebook.error', self._error)

    display(IPython.display.HTML('''
        <label for="query">Buscar: </label>
        <input type="text" id="query" value="Machine Learning+Deep Learning" style="width: 400px;"/>
        <br/>
        <br/>
        
        <label for="fields">Fields: </label>
        <input type="text" id="fields" value="title,abstract,isOpenAccess,fieldsOfStudy" style="width: 400px;"/>
        <br/>
        <br/>
 
        <label for="limit">Limite: </label>
        <input type="text" id="limit" value="10" style="width: 50px;"/><br/>
        <br/>

        <button id='button'>Pesquisar</button>
        <br/>
        <br/>
        <script>
          document.querySelector('#button').onclick = async () => {

            var search = document.getElementById("query").value
            var fields = document.getElementById("fields").value
            var limit = document.getElementById("limit").value

            google.colab.kernel.invokeFunction('notebook.searching', [], {});
            
            var data = await fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=0&limit=${limit}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {
      
      // res.data.map((as, idx) => {
      //   res.data[idx].fieldsOfStudy = as.fieldsOfStudy.join(", ")
      // })
    // .catch(err=>{
    //   google.colab.kernel.invokeFunction('notebook._error', [], {});
    //   })

      return(res)})
            google.colab.kernel.invokeFunction('notebook.AddListItem', [data], {});
          };
        </script>
        '''))
    

  def _error(self):
    print("Deu ruim")

  def _searching(self):
    with output.use_tags('some_outputs'):
      print("\n\nPesquisando...")
      sys.stdout.flush();

  def _add_list_item(self, value):
    output.clear(output_tags='some_outputs')
    data = pd.DataFrame(value['data'])
    
    print(f"Achou {value['total']} papers.")
    print(data.head())

    self.data = data
    




In [ ]:
Resultados = Search()

Achou 648721 papers.
                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
3  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
4  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]

[5 rows x 5 columns]


In [ ]:
# Os dados ficam na variável data
Resultados.data

,paperId,title,abstract,isOpenAccess,fieldsOfStudy
0,846ff7afb7670d62f88b4a8cc99d306ffb81b075,"Artificial Intelligence, Machine Learning, Dee...",This article was presented at the 2017 annual ...,False,[Medicine]
1,5dc53e50148b01fe8b9536eb79fa6b1dce924174,Promising Artificial Intelligence‐Machine Lear...,Abstract: The lifestyle of modern society has ...,False,[Medicine]
2,7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f,Rainfall Prediction using Machine Learning & D...,"In India, Agriculture is the key point for sur...",False,[Computer Science]
3,59444b096f7c8a561d540102e8b5bfb189edabc6,Perspectives on the Impact of Machine Learning...,The fields of machining learning and artificia...,True,None
4,eee313380ccb45807ea0afa3c1df86f6b48b8867,Model Selection and Evaluation for Machine Lea...,Materials processing is a critical subset of m...,False,[Computer Science]
5,46479bbea7749cb2db35b139206039531327053c,Fifty years of computer analysis in chest imag...,"Half a century ago, the term “computer-aided d...",False,[Computer Science]
6,b69fe5a837277ddbea5215d6bacd3a902e9d11ce,Fifty years of computer analysis in chest imag...,"Half a century ago, the term “computer-aided d...",True,[Medicine]
7,b0bf64ccbd651e8c7bc141d8aabaecff562e93a1,"AI, Machine Learning & Deep Learning Risk Mana...",The current paper proposes how model risk mana...,False,[Computer Science]
8,042ab08ec6782cf217f13175162bfd48f7350114,COVID-19 Pandemic: Role of Machine Learning & ...,Introduction: COVID-19 has become a global pan...,False,[Computer Science]
9,03e7832982986159400a8eeab148487ffcfabe56,Machine Learning: Deep Learning,This is the third chapter in the series on mac...,True,[Computer Science]
